In [15]:
import json
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import bootstrap
from collections import defaultdict
import os
from scipy.stats import bootstrap
import torch

from train import get_flops
from config import Embedder, Dataset
from test_from_studies import gen_args_from_optuna_params
from dataset import get_dataset
from model import build_model


In [2]:
result_files = {
    Dataset.PROTEINS: "study_outputs/dataset-PROTEINS/results.json",
    Dataset.ENZYMES: "study_outputs/dataset-ENZYMES/results.json",
}

In [3]:
result_dict = {}
for ds, path in result_files.items():
    with open(path, "r") as f:
        result_dict[ds] = json.load(f)

In [11]:
ds = get_dataset(Dataset.PROTEINS)

In [16]:
test_graph = ds[0]
test_graph.x.shape, test_graph.y.shape, test_graph.edge_index.shape

(torch.Size([42, 4]), torch.Size([1]), torch.Size([2, 162]))

In [18]:
results = []

for utility_fn in result_dict[Dataset.PROTEINS]["best_trials"]["MLP-2^D"].keys():
    trials = result_dict[Dataset.PROTEINS]["best_trials"]["MLP-2^D"][utility_fn]
    flops = []
    for trial in trials:
        args = gen_args_from_optuna_params(
            Dataset.PROTEINS,
            Embedder.MLP_2_D,
            trial["params"]
        )

        model = build_model(args, 4, 2)

        flops.append(get_flops(model, torch.device("cpu"),  test_graph.x, test_graph.edge_index))
    
    res = bootstrap((flops,), np.mean, confidence_level=0.95)
    flop = (res.confidence_interval.low + res.confidence_interval.high) / 2
    error = (res.confidence_interval.high - res.confidence_interval.low) / 2
    results.append({
        "utility_fn": utility_fn,
        "flop": flop,
        "error": error
    })

print(json.dumps(results, indent=4))

Unsupported operator aten::mul encountered 3 time(s)
Unsupported operator aten::sum encountered 2 time(s)
Unsupported operator aten::ne encountered 1 time(s)
Unsupported operator aten::repeat encountered 1 time(s)
Unsupported operator aten::alias encountered 1 time(s)
Unsupported operator aten::index_select encountered 5 time(s)
Unsupported operator aten::add encountered 3 time(s)


Unsupported operator aten::leaky_relu encountered 1 time(s)
Unsupported operator aten::expand_as encountered 4 time(s)
Unsupported operator aten::scatter_reduce_ encountered 2 time(s)
Unsupported operator aten::sub encountered 1 time(s)
Unsupported operator aten::exp encountered 1 time(s)
Unsupported operator aten::scatter_add_ encountered 2 time(s)
Unsupported operator aten::div encountered 1 time(s)
Unsupported operator aten::mean encountered 1 time(s)
Unsupported operator aten::log_softmax encountered 1 time(s)
The following submodules of the model were never called during the trace of the graph. They may be unused, or they were accessed by direct calls to .forward() or via other python methods. In the latter case they will have zeros for statistics, though their statistics will still contribute to their parent calling module.
model._trim, model.act, model.dropout
Unsupported operator aten::index_select encountered 2 time(s)
Unsupported operator aten::new_ones encountered 2 time(s)


[
    {
        "utility_fn": "best_all",
        "flop": 504936.6,
        "error": 423288.6
    },
    {
        "utility_fn": "best_acc",
        "flop": 9582623.4,
        "error": 5262402.6
    },
    {
        "utility_fn": "low_params",
        "flop": 129864.0,
        "error": 62395.200000000004
    }
]
